In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,  LabelEncoder
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [24]:
df = pd.read_csv("D:/credit_card_fraud_dataset.csv")
df.drop(columns=["TransactionID", "TransactionDate"], inplace=True)

In [26]:
for col in df.select_dtypes(include='object').columns:
    df[col] = LabelEncoder().fit_transform(df[col])
X = df.drop("IsFraud", axis=1)
y = df["IsFraud"]

In [28]:
X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

In [30]:
def build_model():
    model = Sequential([
        Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [36]:
model_base = build_model()
model_base.fit(X_train, y_train, epochs=5, batch_size=64, verbose=0)
y_pred_base = (model_base.predict(X_test) >= 0.5).astype(int)
report_base = classification_report(y_test, y_pred_base, output_dict=True, zero_division=0)

C:\Users\indum\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [38]:
X_res, y_res = SMOTE(random_state=42).fit_resample(X_train, y_train)

In [40]:
model_smote = build_model()
model_smote.fit(X_res, y_res, epochs=5, batch_size=64, verbose=0)
y_pred_smote = (model_smote.predict(X_test) >= 0.5).astype(int)
report_smote = classification_report(y_test, y_pred_smote, output_dict=True)

C:\Users\indum\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [42]:
print("\n🔹 Precision / Recall / F1 Comparison:\n")
print("{:<12} {:<10} {:<10} {:<10}".format("Model", "Precision", "Recall", "F1-score"))
for report, label in zip([report_base, report_smote], ['Baseline', 'SMOTE']):
    p = report['1']['precision']
    r = report['1']['recall']
    f1 = report['1']['f1-score']
    print(f"{label:<12} {p:<10.4f} {r:<10.4f} {f1:<10.4f}")


🔹 Precision / Recall / F1 Comparison:

Model        Precision  Recall     F1-score  
Baseline     0.0000     0.0000     0.0000    
SMOTE        0.0098     0.3700     0.0192    
